In [1]:
from pyspark.sql import SparkSessions


In [4]:
import pyspark

In [7]:
spark = SparkSession.builder.getOrCreate()
strings=spark.read.text('../README.md')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/08 19:39:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
strings.schema

StructType([StructField('value', StringType(), True)])

## Time evolution (max diff between start and end)

# Some stuff

In [ ]:
for (dh,dw,g_id,dom) in [(360, 640, 333372, 'ID_9032')]:
    print((dh,dw,g_id,dom))
    gid= g_id
    
    example = data_heatmap_ts[(data_heatmap_ts.grid_id==gid) & (data_heatmap_ts.domain==dom)
                    & (data_heatmap_ts.display_height==dh) & (data_heatmap_ts.display_width==dw)].copy().reset_index()
    
    example = example.sort_values(by='timestamp').reset_index()
    example['click_x1'] = (example.click_x/dw*(dw//5)).astype(int)
    example['click_y1'] = (example.click_y/dh*(dh//5)).astype(int)
    print('click number',example.clicks.sum())
    
    plt.axhline(y=-4.540, color='red', linestyle='--')
    rolling[(dh,dw,g_id,dom)].plot(linewidth = '.75')
    rolling[(dh,dw,g_id,dom)].rolling(window=120).mean().shift(-60).plot(linewidth = '2.5')
    plt.grid()
    plt.title(f'Evolution of LR by timestamped clicks')
    plt.xlabel('click number')
    plt.ylabel('LR')
    # plt.savefig(f'asdf')

    
    ex = np.arange(0, example.shape[0], 100, dtype=int)

    nr_pics = example.clicks.sum()//200+1
    fig, axes = plt.subplots(1,nr_pics, figsize=(nr_pics*dw//50,dh//50)) 
    fig.suptitle(f'Grid = {gid}, Dom = {dom}, (height,width): {dh,dw}')
    
    i, j = 0, 3
    
    for ax_idx, ax in enumerate(axes.flat):
        if j-1 < ex.shape[0]:
            cli = ex[i]
            if j >= len(ex):
                clj = example.shape[0]
            else:
                clj = ex[j]
            i += 2
            j += 2
            
            click_matrix = example.iloc[cli:clj].groupby(['click_x1', 'click_y1']).clicks.sum().reset_index().pivot(
                "click_y1", "click_x1", "clicks").fillna(0)
    
            sns.heatmap(click_matrix, cmap="YlGnBu", 
                        annot=False, cbar=True, fmt="g", ax=ax)
            ax.set_title(f'Clicks {cli} to {clj}')
            ax.set_xlabel('x bin')
            ax.set_ylabel('y bin')
            ax.invert_yaxis()
            
        else:
            ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(f'asdf_1')
    plt.show()


In [ ]:
for (dh,dw,g_id,dom) in [(360, 640, 333372, 'ID_9032')]:
    print((dh,dw,g_id,dom))
    gid= g_id
    
    example = data_heatmap_ts[(data_heatmap_ts.grid_id==gid) & (data_heatmap_ts.domain==dom)
                    & (data_heatmap_ts.display_height==dh) & (data_heatmap_ts.display_width==dw)].copy().reset_index()
    example = example.sort_values(by='timestamp').reset_index()
    example['click_x1'] = (example.click_x/example.display_width*50).astype(int)
    example['click_y1'] = (example.click_y/example.display_height*50).astype(int)

    ex = np.arange(0, example.shape[0], 100, dtype=int)

    i, j = 0, 2
    
    for k in range(example.clicks.sum()//100):    
        cli = ex[i]
        if j >= len(ex):
            clj = example.shape[0]
        else:
            clj = ex[j]
        i += 1
        j += 1
        
        click_matrix = example.iloc[cli:clj].groupby(['click_x1', 'click_y1']).clicks.sum().reset_index().pivot(
            "click_y1", "click_x1", "clicks").fillna(0)

        plt.figure(figsize=(dh//50,dw//50))
        sns.heatmap(click_matrix, 
                    cmap = "YlGnBu", 
                    annot = False, 
                    cbar = True, 
                    fmt = "g")
        plt.title(f'(grid_id, domain, dh, dw): ({g_id}, {dom}, {dh}, {dw}), Click Nr: {example.clicks.sum()}')
        plt.xlabel('x bin')
        plt.ylabel('y bin')
        plt.gca().invert_yaxis() 
        plt.savefig(f'asdf_{k}')
        
        plt.show()